<a href="https://colab.research.google.com/github/Gainward777/Point_Cloud_Single_Photo/blob/main/PointCloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ImageFromDepth

In [ ]:
#@title Installs and imports
!pip install open3d
!pip install timm
!pip install pyheif  

##Load libs

In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import cv2
import os

import torch
import matplotlib.image as pltim
from PIL import Image
import pyheif 
import shutil
from google.colab import files
import math

##Get Depth (MiDaS)

In [ ]:
#@title Load Model
model_type = "DPT_Large"  # @param ["DPT_Large", "DPT_Hybrid", "MiDaS_small"] 
                             # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

In [3]:
#@title HEIC convert function
def convert_heic(path):
    new_path=f"{path.split('.')[0]}.jpg"
    heif_file = pyheif.read(path)
    img=Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
    img.save(new_path)
    return new_path

In [ ]:
#@title Load files if it's on desctop
from google.colab import files

dwld_directory = '/content/origins' #@param {type:"string"}
if dwld_directory.split('/')[-1] != '':
    dwld_directory=dwld_directory+'/'
if not os.path.exists(dwld_directory):
   os.mkdir(dwld_directory)
%cd {dwld_directory}

uploaded = files.upload()

up_pathes=[]
up_list=list(uploaded.keys())
for name in up_list:
    ext=name.split('.')[-1]
    if ext=='HEIC' or ext == 'heic':
        path=convert_heic(f"{dwld_directory}{name}")
        up_pathes.append(path)
    else:
        up_pathes.append(f"{dwld_directory}{name}")

In [ ]:
#@title Add directory if files on drive
directory = "/content/drive/images" #@param {type:"string"}
file_list=os.listdir(directory)
up_pathes=[]
for f in file_list:
    ext=name.split('.')[-1]
    if ext=='HEIC' or ext == 'heic':
        path=convert_heic(f"{directory}/{f}")
        up_pathes.append(path)
    else:
        up_pathes.append(f"{directory}/{f}")

###Resize if it's necessary

In [10]:
def normalize_size(img, height, width, round_h_coef=None, round_w_coef=None):    
    true_h_coef=img.shape[0]/height
    if round_h_coef==None:
        round_h_coef=rounding_down(true_h_coef) #int((img.shape[0]/480)*10)/10
    true_w_coef=img.shape[1]/width
    if round_w_coef == None:
        round_w_coef=rounding_down(true_w_coef)
    start_h=int((true_h_coef*height-round_h_coef*height)/4)
    end_h=int(round_h_coef*height+start_h)
    start_w=int((true_w_coef*width-round_w_coef*width)/4)
    end_w=int(round_h_coef*width+start_w)
    return img[start_h:end_h,start_w:end_w], round_h_coef, round_w_coef

def rounding_down(value_for_round: float, round_size: int=10):
    return int(value_for_round*round_size)/round_size

def resize_image(im_path, height, width):
    img = cv2.imread(im_path, cv2.IMREAD_UNCHANGED)
    print('Image :', im_path)
    print('Original Dimensions : ',img.shape)
    img, round_h_coef, round_w_coef=normalize_size(img, height, width)
    if round_h_coef>round_w_coef:
        round_h_coef=round_w_coef
    img, round_h_coef, round_w_coef=normalize_size(img, height, width, round_h_coef, round_h_coef)

    img = cv2.resize(img, (width,height))
    print('New Dimensions : ', img.shape)
    os.remove(im_path)
    cv2.imwrite(im_path, img) 

In [11]:
for path in up_pathes:
    resize_image(path, 480, 640)

Image : /content/origins/20 (1).jpg
Original Dimensions :  (2736, 3648, 3)
New Dimensions :  (480, 640, 3)
Image : /content/origins/37 (1) (1).jpg
Original Dimensions :  (1704, 2272, 3)
New Dimensions :  (480, 640, 3)
Image : /content/origins/85 (1).jpg
Original Dimensions :  (2736, 3648, 3)
New Dimensions :  (480, 640, 3)
Image : /content/origins/High_quality_nude_1.0_adult_1.5_maid_1.5_beautiful_big_tits_skinny_photo_full_face_bare_1894038054 (1).jpg
Original Dimensions :  (1280, 1024, 3)
New Dimensions :  (480, 640, 3)
Image : /content/origins/IMG_6376.jpg
Original Dimensions :  (480, 640, 3)
New Dimensions :  (480, 640, 3)
Image : /content/origins/IMG_6601.jpg
Original Dimensions :  (480, 640, 3)
New Dimensions :  (480, 640, 3)
Image : /content/origins/IMG_6750.jpg
Original Dimensions :  (480, 640, 3)
New Dimensions :  (480, 640, 3)
Image : /content/origins/IMG_7185.jpg
Original Dimensions :  (480, 640, 3)
New Dimensions :  (480, 640, 3)
Image : /content/origins/IMG_7750.jpg
Origin

###Load transforms and apply it to images

In [12]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


In [13]:
transformed_imgs=[]
for path in up_pathes:
    img = cv2.imread(path)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    transformed_imgs.append(transform(img))

#input_batch = transform(img).to(device)

###Make depths

In [14]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

outputs=[]
for input in transformed_imgs:
    with torch.no_grad():
        prediction = midas(input.to(device))

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    outputs.append(prediction.cpu().numpy())

In [15]:
#@title Save results
depth_pathes=[]

depth_directory = '/content/depths' #@param {type:"string"}
if depth_directory.split('/')!='':
    depth_directory=depth_directory+'/'
if not os.path.exists(depth_directory):
   os.mkdir(depth_directory)
%cd {depth_directory}

for i in range(len(up_list)):
    path=f"{depth_directory}/{up_list[i].split('.')[0]}_depth.jpg"
    depth_pathes.append(path)
    pltim.imsave(path, outputs[i])

/content/depths


##Reconstruction

In [ ]:
#@title Load files (if they were not loaded in the previous step)
from google.colab import files

dwld_directory = '/content/origins' #@param {type:"string"}
if dwld_directory.split('/')[-1] != '':
    dwld_directory=dwld_directory+'/'
if not os.path.exists(dwld_directory):
   os.mkdir(dwld_directory)
%cd {dwld_directory}

uploaded = files.upload()

up_pathes=[]
up_list=list(uploaded.keys())
for name in up_list:
    ext=name.split('.')[-1]
    if ext=='HEIC' or ext == 'heic':
        path=convert_heic(f"{dwld_directory}{name}")
        up_pathes.append(path)
    else:
        up_pathes.append(f"{dwld_directory}{name}")
    #up_pathes.append(f"/content/origins/{name}")

depth_directory = '/content/depths' #@param {type:"string"}
if depth_directory.split('/')!='':
    depth_directory=depth_directory+'/'
if not os.path.exists(depth_directory):
   os.mkdir(depth_directory)
%cd {depth_directory}

depth_uploaded = files.upload()

depth_pathes=[]
depth_list=list(depth_uploaded.keys())
for name in depth_list:
    depth_pathes.append(f"{depth_directory}{name}")

In [ ]:
#@title Load only depths (if they were not loaded in the previous step)
depth_directory = '/content/depths' #@param {type:"string"}
if depth_directory.split('/')!='':
    depth_directory=depth_directory+'/'
if not os.path.exists(depth_directory):
   os.mkdir(depth_directory)
%cd {depth_directory}

depth_uploaded = files.upload()

depth_pathes=[]
depth_list=list(depth_uploaded.keys())
for name in depth_list:
    depth_pathes.append(f"{depth_directory}{name}")

In [76]:
#@title Get pointclouds
ply_directory = '/content/3d' #@param {type:"string"}
pcd_dict={}
if not os.path.exists(ply_directory):
   os.mkdir(ply_directory)
#all files need to be .jpg
for i in range(len(up_pathes)):
    #open files
    color_raw = o3d.io.read_image(up_pathes[i])
    depth_raw = o3d.io.read_image(depth_pathes[i])
    #create geometry
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw)
    #get camera
    camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)
    o3d.io.write_point_cloud(f"{ply_directory}/{up_pathes[i].split('/')[-1].split('.')[0]}.ply", pcd)

In [7]:
#@title Download

def download_folder(folder_path):    
    path=shutil.make_archive(folder_path.split('/')[-1], 'zip', folder_path)
    files.download(path)

%cd /content
folders=[]
ply_dir = False #@param {type:"boolean"} 
depth_dir=False #@param {type:"boolean"} 
dwld_dir=True #@param {type:"boolean"} 
if ply_dir: folders.append(ply_directory)
if depth_dir: folders.append(depth_directory[:-1])
if dwld_dir: folders.append(dwld_directory[:-1])

for path in folders:
    download_folder(path)

/content


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>